<h2>Centerline</h2>

In this project, we use a centerline during the training phase to help the agent to avoid walls and to follow the track.

One way of defining such a trajectory is to make a human play the track safetly and save car's data at regular timesteps.

We also use states of the game to rewind the car at a random positions on the track.

In [113]:
import os
import time
import pickle
import keyboard
import numpy as np

import tminterface as tmi
from tminterface.interface import TMInterface, Client

In [3]:
# Instantiate the Interface
# The client is used to connect the interface to the TMI server
# The interface runs itself in a separate thread

interface = TMInterface()
client = Client()

interface.register(client)

True

In [110]:
state = interface.get_simulation_state()

In [111]:
# HELPERS 

# Dynamic state of the car. Position, rotation, speed, etc...
#print(state.dyna.current_state)

# Data about wheels (index 1 out of 4). Contact with the ground, sliding...
#print(state.simulation_wheels[0])

# Time in milisseconds
#print(state.time / 1000 / 60)

[HmsDynaStateStruct object at 0x2007ed2dae0]
quat (ndarray): 
	[0.04067126289010048 -4.430955414136406e-06 -0.9991726875305176
	 8.828952502426546e-08]
rotation (ndarray): 
	[[-0.9966920614242554 8.847398021316621e-06 -0.08127523213624954]
	 [8.86176076164702e-06 1.0 1.8399214241071604e-07]
	 [0.08127523213624954 -5.368580673348333e-07 -0.9966920614242554]]
position (ndarray): [611.6317749023438 9.358945846557617 597.2733154296875]
linear_speed (ndarray): [-0.008326033130288124 -2.932654297183035e-06 0.0006789963226765394]
add_linear_speed (ndarray): [0.0 0.0 0.0]
angular_speed (ndarray): [7.07295403117314e-05 -2.4981636204124413e-10 0.0008842874085530639]
force (ndarray): [-1.6655386686325073 0.00011350531713105738 0.13581641018390656]
torque (ndarray): [0.01205363031476736 -2.9802322387695312e-08 0.14735575020313263]
inverse_inertia_tensor (ndarray): 
	[[0.4847564399242401 -1.0767983837922657e-08 0.05832459777593613]
	 [-1.0767983837922657e-08 0.48000001907348633 -1.3203610649270558e

In [62]:
TRACK_DATA_PATH = "track_data/"
track_name = "A01"

def create_folder_path(track_name):
    
    track_folder = os.path.join(TRACK_DATA_PATH, track_name)
    
    if os.path.exists(track_folder):
        n_run = len(os.listdir(track_folder))
    else:
        os.makedirs(track_folder)
        n_run = 0
    
    run_folder = os.path.join(track_folder, 'run-%s' % (n_run + 1))
    
    os.makedirs(run_folder)
    
    return run_folder

In [114]:
"""
    Main Loop used to capture game data
"""

run_folder = create_folder_path(track_name=track_name)

# Time in milisseconds
save_position_every = 1_000
save_state_every = 1_000

state = interface.get_simulation_state()

# Setup initial position and state before player starts to move
positions = [{
    'time': None,
    'position': state.dyna.current_state.position
}]
pickle.dump(state, open(os.path.join(run_folder, f"state_{0:05d}.pkl"), "wb"))
n_saved_states = 1

# Wait for the user to move to start saving data
print("Waiting for player to move")
while np.linalg.norm(state.dyna.current_state.linear_speed) < 0.1:
    state = interface.get_simulation_state()

positions[0]['time'] = state.time

print("Start to record data")

start_time = state.time

while True:
    time.sleep(.1)
    
    if keyboard.is_pressed("q"):
        break
    
    state = interface.get_simulation_state()
    
    if (state.time - start_time) / save_position_every > len(positions):
        positions.append({
            'time': state.time,
            'position': state.dyna.current_state.position
        })
        
    if (state.time - start_time) / save_state_every > n_saved_states:
        pickle.dump(state, open(os.path.join(run_folder, f"state_{n_saved_states:05d}.pkl"), "wb"))
        n_saved_states += 1

pickle.dump(positions, open(os.path.join(run_folder, "positions.pkl"), "wb"))

Waiting for player to move
Start to record data


In [116]:
"""
    Rewind the game to a specific state
"""

state_files = list(filter(lambda x: x.startswith("state"), os.listdir(run_folder)))

for state_file in state_files:
    state = pickle.load(open(os.path.join(run_folder, state_file), "rb"))
    interface.rewind_to_state(state)
    time.sleep(1)

In [117]:
"""
    Load positions
"""

positions = pickle.load(open(os.path.join(run_folder, "positions.pkl"), "rb"))
print(positions[0])

{'time': 5090, 'position': [ArrayFieldProxy object at 0x2007ea3bfd0]
Data: [980.8002319335938 9.356952667236328 656.001220703125]}
